In [15]:
import os
import time
import json
import random
import requests
import platform

import grpc
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

%matplotlib inline

In [2]:
# TF-Serving gRPC informations: https://github.com/tensorflow/serving/blob/master/tensorflow_serving
# Docker informations: https://www.docker.com

In [3]:
# Check OS informations, docker version

resp = !docker --version
docker_version = ' '.join(resp[0].split(' ')[2:])
print("Current OS: \n"
      "    {}\n".format(platform.platform()))
print("Docker version: \n"
      "    {}".format(docker_version))

Current OS: 
    Darwin-18.7.0-x86_64-i386-64bit

Docker version: 
    19.03.8, build afacb8b


In [33]:
# Prepare testing data

mnist = tf.keras.datasets.mnist
(_, _), (x_test, y_test) = mnist.load_data()
x_test = x_test.astype(np.float32)
x_test /= 255.0

In [5]:
# Print MNIST dataset informations
# 打印 MNIST 資料集相關資訊

print("Images numbers:\n"
        "     Test : {}".format(x_test.shape[0]))
print("Image size: {} * {}".format(x_test.shape[1:][0], x_test.shape[1:][1]))

Images numbers:
     Test : 10000
Image size: 28 * 28


In [6]:
# Pull tf-serving latest official image from docker hub 
# Or use !docker pull tensorflow/serving:1.12.0 to pull specific version
# Use gpu with tensorflow/serving:latest-gpu
# 從 docker hub 拉取最新的 tf-serving 映像檔
# 或是使用 !docker pull tensorflow/serving:1.12.0 拉取特定版本
# Gpu 版本請用 tensorflow/serving:latest-gpu

!docker pull tensorflow/serving:latest

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [ ]:
# Run serving docker, better not run this shit in IPython notebook
# Port 8501 exposed for RESTful API, Port 8500 exposed for gRPC
# 請打開一個終端已架設 serving docker，別使用 IPython notebook

!$MODELPATH = "Your models folder path ... must be absolute path!"
!docker run -t --rm -p 8500:8500 \
    --mount type=bind,source=$MODELPATH,target=/models \
    -e MODEL_NAME=mnist \
    --name serving_test \
    tensorflow/serving

In [7]:
# You can get informations of served model with API after you successfully ran a serving docker
# 如果成功跑起一個 serving docker，可以用 API 取得模型資訊

!curl http://localhost:8501/v1/models/mnist

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


In [8]:
# One of the key advantages of REST APIs is that they provide a great deal of flexibility
# Data is not tied to resources or methods, so REST can handle multiple types of calls
# Use python request lib

resp = requests.get('http://localhost:8501/v1/models/mnist')
print(resp.text)

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}



In [9]:
# Get model metadata that we can easily know the signatures in model

resp = requests.get('http://localhost:8501/v1/models/mnist/metadata')
print("Status code: \n"
      "         {}".format(resp))
print("Respones: \n"
      "         {}".format(resp.text))

Status code: 
         <Response [200]>
Respones: 
         {
"model_spec":{
 "name": "mnist",
 "signature_name": "",
 "version": "1"
}
,
"metadata": {"signature_def": {
 "signature_def": {
  "serving_default": {
   "inputs": {
    "flatten_input": {
     "dtype": "DT_FLOAT",
     "tensor_shape": {
      "dim": [
       {
        "size": "-1",
        "name": ""
       },
       {
        "size": "28",
        "name": ""
       },
       {
        "size": "28",
        "name": ""
       }
      ],
      "unknown_rank": false
     },
     "name": "serving_default_flatten_input:0"
    }
   },
   "outputs": {
    "dense_1": {
     "dtype": "DT_FLOAT",
     "tensor_shape": {
      "dim": [
       {
        "size": "-1",
        "name": ""
       },
       {
        "size": "10",
        "name": ""
       }
      ],
      "unknown_rank": false
     },
     "name": "StatefulPartitionedCall:0"
    }
   },
   "method_name": "tensorflow/serving/predict"
  },
  "__saved_model_init_op": {
   "inp

In [36]:
hostport ='localhost:8500'
channel = grpc.insecure_channel(hostport)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
for i in range(1):
    request = predict_pb2.PredictRequest()
    request.model_spec.name ='mnist'
    request.model_spec.signature_name ='serving_default'
    request.inputs['flatten_input'].CopyFrom(tf.make_tensor_proto(x_test[i], shape=[1,1,28, 28]))
    result = stub.Predict(request)
    print(result)
    end = time.time()

outputs {
  key: "dense_1"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: 0.11874900013208389
    float_val: 0.03669405356049538
    float_val: 0.06553603708744049
    float_val: 0.09101864695549011
    float_val: 0.05833970382809639
    float_val: 0.06880989670753479
    float_val: 0.04713801294565201
    float_val: 0.04002722352743149
    float_val: 0.35213395953178406
    float_val: 0.12155339866876602
  }
}
model_spec {
  name: "mnist"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

